# Multi-Agent Support
This is an example implementation of tracking events from two separate agents

In [1]:
import agentops
from agentops.agent import track_agent
from dotenv import load_dotenv
import os
import openai
import logging

from IPython.display import display, Markdown

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', "<your_openai_key>")
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY', "<your_agentops_key>")
logging.basicConfig(level=logging.INFO) # this will let us see that calls are assigned to an agent

In [3]:
ao_client = agentops.Client(AGENTOPS_API_KEY)
openai_client = openai.Client()

INFO:root:View info on this session at https://agentops.ai/dashboard?session_id=e9ecbebe-5aee-4130-b007-895539ac6d96


Now lets create a few agents!

In [4]:
@track_agent(name='qa')
class QaAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a qa engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content
        
@track_agent(name='engineer')
class EngineerAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a software engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content

In [5]:
qa = QaAgent()
engineer = EngineerAgent()

In [6]:
generated_func = engineer.completion("Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:LLM call from agent named: engineer


Unable to parse a chunk for LLM call {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are a software engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.'}], 'temperature': 0.5} - skipping upload to AgentOps


In [7]:
display(Markdown('```python\n' + generated_func + '\n```'))

```python
def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result
```

In [8]:
generated_test = qa.completion("Write a python unit test that test the following function: \n " + generated_func)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:LLM call from agent named: qa


Unable to parse a chunk for LLM call {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are a qa engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python unit test that test the following function: \n def multiply_and_divide(num1, num2):\n    result = (num1 * num2) / 2\n    return result'}], 'temperature': 0.5} - skipping upload to AgentOps


In [9]:
display(Markdown('```python\n' + generated_test + '\n```'))

```python
import unittest

def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result

class TestMultiplyAndDivide(unittest.TestCase):

    def test_multiply_and_divide(self):
        self.assertEqual(multiply_and_divide(4, 2), 4)
        self.assertEqual(multiply_and_divide(5, 3), 7.5)
        self.assertEqual(multiply_and_divide(0, 10), 0)
        self.assertEqual(multiply_and_divide(-6, 3), -9)

if __name__ == '__main__':
    unittest.main()
```

Lets verify one more thing! If we make an LLM call outside of the context of a tracked agent, we want to make sure it gets assigned to the Default Agent.

In [10]:
res = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are not a tracked agent"},
    {"role": "user", "content": "Say hello"}]
)
res.choices[0].message.content

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Unable to parse a chunk for LLM call {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are not a tracked agent'}, {'role': 'user', 'content': 'Say hello'}]} - skipping upload to AgentOps


'Hello! How can I assist you today?'

You'll notice that we didn't log an agent name, so the AgentOps backend will assign it to the Default Agent for the session!